In [43]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from scipy.stats import norm
from scipy.stats import linregress
from tqdm import tqdm

In [44]:
# dataname = "V1_Mouse_Brain_Sagittal_Posterior_Section_2"
dataname = "V1_Mouse_Brain_Sagittal_Posterior"

# dataname = "V1_Mouse_Brain_Sagittal_Anterior_Section_2"
# dataname = "V1_Mouse_Kidney"
samples = pd.read_csv("output/make_samplesheet/spatial.csv",index_col = 0)
# samples
row = samples.loc[dataname]

# score = "ReadZS"
# score2 = "ReadZS_ge"
# thresh = 1000

score = "SpliZ"
score2 = "ge"
thresh = 5

scores = pd.read_csv("output/make_samplesheet/scores.csv",index_col=0)
srow = scores.loc[score]
srow2 = scores.loc[score2]

In [45]:
df = pd.read_csv(row[srow["valname"]],sep="\t")
df2 = pd.read_csv(row[srow2["valname"]],sep="\t")
df["num_spots"] = df[srow["genecol"]].map(df.groupby(srow["genecol"])[srow["cellid"]].nunique())
df = df[df["num_spots"] > thresh]
df["gene_cell"] = df[srow["genecol"]] + "_" + df[srow["cellid"]]
df2["gene_cell"] = df2[srow2["genecol"]] + "_" + df2[srow2["cellid"]]
df[srow2["col"]] = df["gene_cell"].map({x : y for x, y in zip(df2["gene_cell"],df2[srow2["col"]])})
df = df[(~(df[srow["col"]].isna())) & (~(df[srow2["col"]].isna()))]

In [46]:
q = 10
cols = [srow["col"],srow2["col"]]
out = {srow["genecol"] : [], "num_spots" : [], "r_squared" : []}

res_map = {}
count = 0
for window, windf in tqdm(df.groupby(srow["genecol"])):
  count += 1
#   if window == "chr8_11013_plus":
  for col in cols:
    qval = q
    
    # quantile
    while True:
      try:
        windf["{}_quant".format(col)] = pd.qcut(windf[col],qval,labels=False)

        break
      except:
        qval -= 1

    # inverse cdf to normal
    windf["{}_norm".format(col)] = norm.ppf((windf["{}_quant".format(col)] + 0.5)/(windf["{}_quant".format(col)].max() + 1))
  
  # perform regression on these values
  result = linregress(windf["{}_norm".format(srow["col"])], windf["{}_norm".format(srow2["col"])])
  out[srow["genecol"]].append(window)
  out["num_spots"].append(windf[srow["cellid"]].nunique())
  out["r_squared"].append(result.rvalue**2)
  windf["predict"] = result.intercept + result.slope*windf["{}_norm".format(srow2["col"])]
  windf["res"] = windf["{}_norm".format(srow["col"])] - windf["predict"]

  res_map = {**res_map,**{x : y for x, y in zip(windf["gene_cell"],windf["res"])}}
#   if count == 2:
#     break
pd.DataFrame.from_dict(out)

  0%|          | 0/178 [00:00<?, ?it/s]/home/users/jolivier/.local/lib/python3.9/site-packages/scipy/stats/_stats_mstats_common.py:170: RuntimeWarning: invalid value encountered in double_scalars
  slope = ssxym / ssxm
/home/users/jolivier/.local/lib/python3.9/site-packages/scipy/stats/_stats_mstats_common.py:187: RuntimeWarning: divide by zero encountered in double_scalars
  slope_stderr = np.sqrt((1 - r**2) * ssym / ssxm / df)
/home/users/jolivier/.local/lib/python3.9/site-packages/scipy/stats/_stats_mstats_common.py:194: RuntimeWarning: invalid value encountered in double_scalars
  intercept_stderr = slope_stderr * np.sqrt(ssxm + xmean**2)
100%|██████████| 178/178 [00:03<00:00, 50.96it/s]


,gene,num_spots,r_squared
0,Alkbh7,16,0.047657
1,Atox1,702,0.001185
2,Atp5c1,1858,0.005919
3,Atp5d,51,0.000802
4,Atp5h,20,0.045685
...,...,...,...
173,Uqcr10,858,0.017769
174,Uqcr11,168,0.029527
175,Uqcrh,29,0.021244
176,Uqcrq,1993,0.000360


In [40]:
df[~df["res"].isna()]["gene"].value_counts()

Rps24      3246
Rpl9       3161
Ptgds      3119
Tpt1       3089
Rpl11      2987
           ... 
Mrpl23      143
Selenoh     127
Carmil2     126
Ndufa4      126
Igsf8       112
Name: gene, Length: 100, dtype: int64

In [29]:
windf

,f2,f1,svd_z_sumsq,dummy,both_ann,barcode,plot_ycoord,array_row,svd_z1,xcoord,...,scZ,num_spots,gene_cell,frac_count,res,scZ_quant,scZ_norm,frac_count_quant,frac_count_norm,predict
53696,0.04751,0.129134,4.699503,NaN,True,TATGTCAAGACCGACT-1,-3815.0,20.0,0.404257,3815.0,...,0.167997,1923,Atp5c1_V1_Mouse_Brain_Sagittal_Posterior_TATGT...,0.000871,0.004005,4,-0.125661,9,1.644854,-0.129666
53697,0.04751,0.129134,1.179929,NaN,True,CGGACGTTACTTGAAG-1,-6451.0,42.0,0.185388,6451.0,...,0.411403,1923,Atp5c1_V1_Mouse_Brain_Sagittal_Posterior_CGGAC...,0.000767,1.774520,9,1.644854,9,1.644854,-0.129666
53698,0.04751,0.129134,1.179929,NaN,True,TATGTAGAAACCCGGC-1,-8727.0,61.0,0.185388,8727.0,...,0.102484,1923,Atp5c1_V1_Mouse_Brain_Sagittal_Posterior_TATGT...,0.000443,-0.373645,3,-0.385320,5,0.125661,-0.011675
53699,0.04751,0.129134,0.392455,NaN,True,ATGACGCGTTCTATCC-1,-7649.0,52.0,0.598278,7649.0,...,-0.534902,1923,Atp5c1_V1_Mouse_Brain_Sagittal_Posterior_ATGAC...,0.000321,-1.086904,1,-1.036433,2,-0.674490,0.050470
53700,0.04751,0.129134,1.179929,NaN,True,ATAAGTTACCGCGACG-1,-6930.0,46.0,0.185388,6930.0,...,0.382570,1923,Atp5c1_V1_Mouse_Brain_Sagittal_Posterior_ATAAG...,0.000446,1.656529,9,1.644854,5,0.125661,-0.011675
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55614,0.04751,0.129134,1.179929,NaN,True,ATTCGACGCCGGGCCT-1,-5732.0,36.0,0.185388,5732.0,...,0.349619,1923,Atp5c1_V1_Mouse_Brain_Sagittal_Posterior_ATTCG...,0.000413,1.028589,8,1.036433,4,-0.125661,0.007844
55615,0.04751,0.129134,1.179929,NaN,True,TGGAAGAAGGGAACGT-1,-6091.0,39.0,0.185388,6091.0,...,0.020105,1923,Atp5c1_V1_Mouse_Brain_Sagittal_Posterior_TGGAA...,0.000409,-0.682334,2,-0.674490,4,-0.125661,0.007844
55616,0.04751,0.129134,22.813822,NaN,True,TAGCGTTGGGTCTTAC-1,-5971.0,38.0,1.113432,5971.0,...,-2.401692,1923,Atp5c1_V1_Mouse_Brain_Sagittal_Posterior_TAGCG...,0.000379,-1.672865,0,-1.644854,3,-0.385320,0.028011
55617,0.04751,0.129134,1.260326,NaN,True,CTCACTTGGCTGGTAA-1,-6211.0,40.0,0.196757,6211.0,...,0.348923,1923,Atp5c1_V1_Mouse_Brain_Sagittal_Posterior_CTCAC...,0.000295,0.957852,8,1.036433,1,-1.036433,0.078581


In [39]:
df["res"] = df["gene_cell"].map(res_map)

In [41]:
df

,f2,f1,svd_z_sumsq,dummy,both_ann,barcode,plot_ycoord,array_row,svd_z1,xcoord,...,plot_xcoord,n.g_End,f0,ontology,svd_z0,scZ,num_spots,gene_cell,frac_count,res
446,7.578857e-31,0.014304,0.066008,NaN,True,TATTAACCTGACCGCG-1,-8248.0,57.0,0.013534,8248.0,...,4531.0,NaN,0.985696,null7.0,-0.256564,-0.249647,171,Ndufb3_V1_Mouse_Brain_Sagittal_Posterior_TATTA...,0.000298,-0.637355
447,7.578857e-31,0.014304,17.507820,NaN,True,ACGGCCAACATGGACT-1,-7768.0,53.0,0.247632,7768.0,...,5219.0,6.0,0.985696,null2.0,4.176901,3.929931,171,Ndufb3_V1_Mouse_Brain_Sagittal_Posterior_ACGGC...,0.000328,0.687610
448,7.578857e-31,0.014304,0.066008,NaN,True,TCTGTTACCCAGCATA-1,-7529.0,51.0,0.013534,7529.0,...,3842.0,NaN,0.985696,null2.0,-0.256564,-0.249647,171,Ndufb3_V1_Mouse_Brain_Sagittal_Posterior_TCTGT...,0.000373,-0.703805
449,7.578857e-31,0.014304,0.066008,NaN,True,TAAGAGGGACAGGGAC-1,-8967.0,63.0,0.013534,8967.0,...,2740.0,NaN,0.985696,null7.0,-0.256564,-0.249647,171,Ndufb3_V1_Mouse_Brain_Sagittal_Posterior_TAAGA...,0.000217,-0.556770
450,7.578857e-31,0.014304,0.066008,NaN,True,CGCGCAAATGTCCAGA-1,-8128.0,56.0,0.013534,8128.0,...,3085.0,NaN,0.985696,null6.0,-0.256564,-0.249647,171,Ndufb3_V1_Mouse_Brain_Sagittal_Posterior_CGCGC...,0.000526,-0.829968
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142521,1.220214e-29,0.031992,0.007518,NaN,True,TCCGAATGGTCCTGAG-1,-5133.0,31.0,0.066777,5133.0,...,9213.0,6.0,0.968008,null0.0,-0.055308,-0.100121,2108,Rpl10_V1_Mouse_Brain_Sagittal_Posterior_TCCGAA...,0.000382,0.663898
142522,1.220214e-29,0.031992,0.007518,NaN,True,AGGCATTGTCGTAGGG-1,-7289.0,49.0,0.066777,7289.0,...,8249.0,6.0,0.968008,null1.0,-0.055308,-0.100121,2108,Rpl10_V1_Mouse_Brain_Sagittal_Posterior_AGGCAT...,0.000617,0.684933
142523,1.220214e-29,0.031992,0.007518,NaN,True,GATAGGATTAATTACA-1,-6211.0,40.0,0.066777,6211.0,...,10108.0,6.0,0.968008,null0.0,-0.055308,-0.100121,2108,Rpl10_V1_Mouse_Brain_Sagittal_Posterior_GATAGG...,0.000243,0.654896
142524,1.220214e-29,0.031992,0.007518,NaN,True,ACGTTCGCAATCAATT-1,-5372.0,33.0,0.066777,5372.0,...,9213.0,6.0,0.968008,null0.0,-0.055308,-0.100121,2108,Rpl10_V1_Mouse_Brain_Sagittal_Posterior_ACGTTC...,0.000193,0.654896
